In [44]:
from kiteconnect import KiteConnect
import datetime as dt
from time import sleep
import pytz
import numpy as np
import pandas as pd
import sklearn.preprocessing as skp
import json
from helper import *
from keras.models import load_model
from pythonLib.layer_utils import AttentionLSTM
from pythonLib.helper import *

# Log in to Kite
vals = json.load(open('config.json')) # read the config
kite = KiteConnect(api_key=vals['API_KEY']) #

print("Here")              
try:
    user = kite.generate_session(vals['REQ_TOKEN'], api_secret=vals['API_SECRET'])
except Exception as e:
    print("Authentication failed", str(e))
    raise

print(user["user_id"], "has logged in") # connected to API                  

instFile = "instruments.csv" # location of the instrument list for current instruments
curInstList = "tradeList.txt" # location of all instruments currently being traded
spreadsFile = "spreads.txt" 
spreadList = pd.read_csv(spreadsFile,header=None).values
stockList = [] # list

with open (curInstList) as f: #populate list of all current stocks
    i = 0
    for each_csv in f:
        each_csv = each_csv.rstrip('\n') # read csv
        curTicker = each_csv # store ticker
        stockList.append(curTicker)
        i+=1
        if i > 2: # first 3 stocks for now
            break
print("Here")
buyModels = [] # list of buy models
sellModels = [] # list of sell Models
instTokens = [] # All the instrument tokens
lags = []   #all the lags
historiesPrices = [] # shape = (stock,cost,vols)
historiesVols = []

#Load some of the bsaics
for i,curStock in enumerate(stockList):
    # print(curStock)
    buyModel = load_model('modelsFin/%sbuyModel.h5' % curStock, custom_objects={'AttentionLSTM': AttentionLSTM}) # load the buy model
    sellModel = load_model('modelsFin/%ssellModel.h5' % curStock, custom_objects={'AttentionLSTM': AttentionLSTM})
    currLag = int(buyModel.layers[0].input.shape[1]) # get how much lag is being used based on the input to the model
    curInstr = findInstToken(curStock, instFile)
    buyModels.append(buyModel)
    sellModels.append(sellModel)
    instTokens.append(curInstr)
    lags.append(currLag)
    historiesPrices.append(np.zeros(currLag))
    historiesVols.append(np.zeros(currLag))
print(instTokens)
# curVol = np.zeros(len(

Here
ZN9243 has logged in
Here
[3463169, 2933761, 3375873]


In [45]:
print(user["user_id"], "has logged in") # connected to API



ZN9243 has logged in


In [4]:
def buyOrd(kiteCli,tSymbol,price,sqVal,stpVal,quant):
    order = kiteCli.place_order(tradingsymbol = tSymbol,
                                    exchange = kite.EXCHANGE_NSE,
                                    quantity = int(quant),
                                    transaction_type = kite.TRANSACTION_TYPE_BUY,
                                    product = kite.PRODUCT_MIS,
                                    order_type =  kite.ORDER_TYPE_LIMIT,
                                    price = float(price),
                                    squareoff = sqVal,
                                    stoploss =  stpVal,
                                    variety = kite.VARIETY_BO,
                                    validity = kite.VALIDITY_DAY,
                                    disclosed_quantity = int(quant/10))
    return order

def sellOrd(kiteCli,tSymbol,price,sqVal,stpVal,quant):
    order = kiteCli.place_order(tradingsymbol = tSymbol,
                                    exchange = kite.EXCHANGE_NSE,
                                    quantity = int(quant),
                                    transaction_type = kite.TRANSACTION_TYPE_SELL,
                                    product = kite.PRODUCT_MIS,
                                    order_type = kite.ORDER_TYPE_LIMIT,
                                    squareoff = sqVal,
                                    stoploss = stpVal,
                                    variety = kite.VARIETY_BO,
                                    price = float(price),
                                    validity = kite.VALIDITY_DAY,
                                    disclosed_quantity = int(quant/10))
    return order



In [75]:
def placeOrder(kiteCli,instToken,bMod,sMod,curStock,lag,spreads):
    """kiteCli = kite client
    historiesPrices = price history
    historiesVols = volume history
    curStock = trading Symbol of current stock
    lag = lag of stock
    spreads = list of [sqVal,stpVal,quant]
    """
    history = kiteCli.historical_data(instToken, 
                                      str(dt.datetime.now().date() - dt.timedelta(days=1)),
                                      str(dt.datetime.now().date() + dt.timedelta(days=1)), "minute", continuous=False)

    curr = h[-45:]
    historiesPrices = np.array([x['close'] for x in curr])
    historiesVols = np.array([x['volume'] for x in curr] )
    print(dt.datetime.now())
    print(historiesPrices)
    print(historiesVols)
    close = skp.minmax_scale(historiesPrices)
    vols = skp.minmax_scale(historiesVols)
    data = np.zeros((1,lag,2))
    data[0,:,0] = close
    data[0,:,1] = vols
    buyProb = bMod.predict([data,data])[0][0] 
    sellProb = sMod.predict([data,data])[0][0]
    # print(spreads)
    sqVal = spreads[0]
    stpVal = spreads[1]
    quant = spreads[2]
    bHigh = spreads[3]
    bLow = spreads[4]
    sHigh = spreads[5]
    sLow = spreads[6]
    print("BuyProb = %.2f Sellprob = %.2f" % (buyProb,sellProb))
    if buyProb > bHigh and sellProb < bLow: # if buy probability is greater than 0.6
        print("Buyprob greater than %.2f at %.2f and SellProb less than %.2f at %.2f" % (bHigh,buyProb,bLow,sellProb))
        print("Buying")
        orderId =  buyOrd(kiteCli,curStock,historiesPrices[-1],sqVal,stpVal,quant) # place a buy order

    elif sellProb > sHigh and buyProb < sLow:
        print("SellProb greater than %.2f at %.2f and Buyprob less than %.2f at %.2f" % (sHigh,sellProb,sLow,buyProb))
        print("Selling  ")
        orderId =  sellOrd(kiteCli,curStock,historiesPrices[-1],sqVal,stpVal,quant) # place a sell order
    else:
        print("No probabilities greater than thresholds, skipping")
    sleep(1)
    return history

In [76]:
while int(dt.datetime.now(pytz.timezone('Asia/Kolkata')).hour) < 15: # Last order goes in at 2 PM
    spreadList = pd.read_csv(spreadsFile,header=None).values # Maybe not needed every minute, we'll see
    for i,curStock in enumerate(stockList):
        print(curStock)
        h = placeOrder(kite,instTokens[i],
                   buyModels[i],sellModels[i],curStock,lags[i],spreadList[i])
    break

GMRINFRA
2018-02-21 23:31:52.786661
[ 27.85  27.9   27.85  27.9   27.9   27.9   27.85  27.9   27.85  27.85
  27.85  27.9   27.85  27.85  27.85  27.85  27.85  27.9   27.85  27.85
  27.95  27.95  27.95  27.95  28.    27.95  28.1   28.15  28.15  28.2
  28.15  28.1   28.1   28.1   28.1   28.1   28.1   28.05  28.05  28.1   28.1
  28.05  28.05  28.05  28.15]
[ 89566  22895  28590  23150  14771  59159 234164  69679  16897   8325
  14840   5391  88956   5024  19026  14969  12893   1219  75254  12077
 130478  32527  17221 273072 432109 110730 491804 460586 288293 474658
 476541 232075 162229 126792 208175  15187 104330 118641  25415 204688
  48494  58315  99567  51736  71774]
BuyProb = 0.70 Sellprob = 0.74
No probabilities greater than thresholds, skipping


/home/s2c/anaconda3/envs/AlgoTrading/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


JPASSOCIAT
2018-02-21 23:31:54.092927
[ 17.6   17.6   17.55  17.55  17.55  17.55  17.6   17.55  17.6   17.55
  17.6   17.55  17.55  17.6   17.6   17.55  17.55  17.55  17.5   17.5   17.5
  17.5   17.55  17.5   17.5   17.55  17.55  17.5   17.55  17.55  17.55
  17.6   17.6   17.65  17.6   17.65  17.65  17.55  17.6   17.55  17.55
  17.6   17.55  17.55  17.6 ]
[226527  19417   1659  24959   2525  15240   6114   4575   2711    854
  14106  11723   4738   2021   6081 313225  17090   1503   6864   8268
  24215   8090 140407  18990  60490  19774  63460  22299  17265 184283
 138665 145644   4755 322213   6990  19356  13600 181158    704   5339
   3139  12017  40321   4589 158816]
BuyProb = 0.06 Sellprob = 0.21
No probabilities greater than thresholds, skipping
RCOM
2018-02-21 23:31:55.390839
[ 16.55  16.55  16.55  16.5   16.5   16.55  16.55  16.5   16.5   16.55
  16.55  16.5   16.5   16.5   16.55  16.5   16.5   16.5   16.45  16.45
  16.5   16.5   16.5   16.55  16.5   16.5   16.55  16.55  16.5   